In [1]:
import torch as t
from torch import nn
from torch.utils.data import Dataset, DataLoader
import plotly.express as px
from IPython.display import display
import pandas as pd
import numpy as np
import copy
from fancy_einsum import einsum
from dataclasses import dataclass
from tqdm.notebook import tqdm_notebook

from einops import rearrange, reduce, repeat

import utils
import cnn_modules as cm
import transformer_modules as tm

In [2]:
def single_head_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of self-attention (see the "Self-Attention in Detail" section of the Illustrated Transformer).

    With this function, you can ignore masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [3]:
def single_head_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of masked self-attention.

    See "The Decoder Side" section of the Illustrated Transformer for an explanation of masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    batch_size, seq_len, embed_size = Q.shape
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    
    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values
    scores = scores.masked_fill(mask==0, -1e9)
    
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [4]:
def multihead_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
    '''
    Implements multihead masked attention on the matrices Q, K and V.

    Q: shape (batch, seq, nheads*headsize)
    K: shape (batch, seq, nheads*headsize)
    V: shape (batch, seq, nheads*headsize)

    returns: shape (batch, seq, nheads*headsize)
    '''
    Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

    batch_size, seq_len, nheads, headsize = Q.shape
    scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
    scores /= Q.shape[-1] ** 0.5

    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values where mask==0
    scores = scores.masked_fill(mask==0, -1e9)

    scores = t.softmax(scores, dim=-1)
    Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
    Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
    return Z

In [5]:
T = t.randn(2, 10, 4)

In [6]:
multihead_masked_attention(T, T, T, 2)

tensor([[[ 0.6290, -0.6051, -0.2954,  0.7529],
         [ 0.6092, -0.2194,  1.0914, -0.1713],
         [ 1.3159, -0.1302,  1.3785, -1.1988],
         [-0.2991, -0.6833,  0.5806, -0.4275],
         [-2.1713,  0.5318, -0.2257,  0.9441],
         [ 1.0399,  0.0493, -1.5412,  0.3850],
         [-0.4043, -1.3045, -0.2809,  0.2067],
         [-0.5782, -0.0068,  0.9746, -2.0765],
         [ 0.3810, -0.6174, -0.6097,  0.1171],
         [-0.2868,  0.8292, -1.1474, -0.8983]],

        [[-1.2872, -0.9363,  1.2608, -0.7509],
         [ 1.4280,  0.1836,  1.2510, -0.7601],
         [-0.1545, -0.4769, -1.4422,  1.1324],
         [ 0.6196,  0.5221, -0.4440,  0.1374],
         [ 1.0475,  0.6375, -0.2633,  1.3483],
         [ 1.9431,  0.3064, -2.1945, -0.2428],
         [-0.0892,  0.3292,  0.4238,  0.2922],
         [-1.2562, -1.9871, -1.2445, -2.4476],
         [ 0.0880, -0.5141, -1.4046,  1.3445],
         [ 0.7443,  0.8426,  0.0528, -1.4915]]])

In [7]:
class MultiheadMaskedAttention(nn.Module):
    W_QKV: nn.Linear
    W_O: nn.Linear

    def __init__(self, hidden_size: int, num_heads: int):
        super().__init__()
        self.num_heads = num_heads
        self.query_size = int(hidden_size / num_heads)
        self.qkv = cm.Linear(hidden_size, 3*hidden_size)
        self.ff = cm.Linear(hidden_size, hidden_size)

    def multihead_masked_attention(self, Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
        '''
        Implements multihead masked attention on the matrices Q, K and V.

        Q: shape (batch, seq, nheads*headsize)
        K: shape (batch, seq, nheads*headsize)
        V: shape (batch, seq, nheads*headsize)

        returns: shape (batch, seq, nheads*headsize)
        '''
        Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

        batch_size, seq_len, nheads, headsize = Q.shape
        scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
        scores /= Q.shape[-1] ** 0.5

        # create lower-left triangle of ones, including the diagonal
        mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
        # fill with close-to-neg-inf values where mask==0
        scores = scores.masked_fill(mask==0, -1e9)

        scores = t.softmax(scores, dim=-1)
        Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
        Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
        return Z

    def forward(self, x: t.Tensor) -> t.Tensor:
        '''
        x: shape (batch, seq, hidden_size)

        Return: shape (batch, seq, hidden_size)
        '''
        out = self.qkv(x)
        Q, K, V = t.tensor_split(out, 3, dim=-1)

        Z = self.multihead_masked_attention(Q, K, V, self.num_heads)

        return self.ff(Z)

In [8]:
@dataclass(frozen=True)
class TransformerConfig:
    '''Constants used throughout your decoder-only transformer model.'''

    num_layers: int
    num_heads: int
    vocab_size: int
    hidden_size: int
    max_seq_len: int
    dropout: float = 0.1
    layer_norm_epsilon: float = 1e-05

In [9]:
config = TransformerConfig(
    num_layers=4, 
    num_heads=2, 
    vocab_size=500, 
    hidden_size=64,
    max_seq_len=100,
    dropout=0.1)

In [10]:
class MLP(nn.Module):

    def __init__(self, hidden_size, dropout):
        super().__init__()
        self.linear1 = cm.Linear(hidden_size, 4 * hidden_size)
        self.gelu = tm.GELU()
        self.linear2 = cm.Linear(4 * hidden_size, hidden_size)
        self.dropout = tm.Dropout(p=dropout)

    def forward(self, x):
        out = self.gelu(self.linear1(x))
        out = self.dropout(self.linear2(out))
        return out

In [11]:
class DecoderBlock(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.attn = tm.MultiheadMaskedAttention(config.hidden_size, config.num_heads)
        self.lnorm1 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)
        self.mlp = tm.MLP(config.hidden_size, config.dropout)
        self.lnorm2 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)

    def forward(self, x: t.Tensor) -> t.Tensor:
        normed_attn = self.lnorm1(self.attn(x))
        out = normed_attn + x
        normed_mlp = self.lnorm2(self.mlp(out))
        out = normed_mlp + out
        return out


In [12]:
class DecoderOnlyTransformer(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.emb = nn.Embedding(config.vocab_size, config.hidden_size)
        self.pos_enc = tm.PositionalEncoding(config.max_seq_len, config.hidden_size)
        self.dropout = tm.Dropout(p=config.dropout)

        decoders = [DecoderBlock(config) for l in range(config.num_layers)]
        self.decoders = nn.Sequential(*decoders)
        
        self.post_norm = tm.LayerNorm(config.hidden_size)

    def forward(self, x: t.Tensor) -> t.Tensor:
        embedding = self.emb(x.long())
        embedding = self.pos_enc(embedding)
        embedding = self.dropout(embedding)
        embedding = embedding.to(t.float32)

        out = self.decoders(embedding)
        out = self.post_norm(out)

        out = einsum("B S E, V E -> B S V", out, self.emb.weight)

        return out

## Testing Transformer

In [13]:
from torch.utils.data import Dataset

class ReverseNumberDataset(Dataset):
    def __init__(self, seq_len, total_size):
        self.seq_len = seq_len
        self.total_size = total_size

    def __len__(self):
        return self.total_size

    def __getitem__(self, idx):
        text = t.randint(0, 9, (self.seq_len, ))
        label = text.flip(dims=[0])
        sample = (text, label)
        return sample

In [14]:
num_ds = ReverseNumberDataset(6, 1_000_000)
num_ds[5]

(tensor([1, 8, 5, 7, 7, 5]), tensor([5, 7, 7, 5, 8, 1]))

In [15]:
trainloader = DataLoader(num_ds, batch_size=256, shuffle=True)

In [16]:
config = TransformerConfig(
    num_layers=2, 
    num_heads=4, 
    vocab_size=10, 
    hidden_size=128,
    max_seq_len=6,
    dropout=0.1)

In [22]:
from typing import Callable


epochs = 1
loss_fn = nn.CrossEntropyLoss()
batch_size = 256

MODEL_FILENAME = "./w1d2_transformer_digits.pt"
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")

def train_transformer(trainloader: DataLoader, epochs: int, loss_fn: Callable) -> list:
    '''
    Defines a Transformer from our custom modules, and trains it on the reversed digit dataset.
    '''

    model = DecoderOnlyTransformer(config).to(device).train()
    optimizer = t.optim.Adam(model.parameters())
    loss_list = []
    accuracy_list = []

    for epoch in range(epochs):

        progress_bar = tqdm_notebook(trainloader)
        for (x, y) in progress_bar:

            x = x.to(t.float32)
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            print(logits.shape)
            logits = rearrange(logits, 'B S V -> (B S) V')
            print(logits)
            print(y.shape)
            y = rearrange(y, 'B S -> (B S)')


            loss = loss_fn(logits, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_list.append(loss.item())

            with t.inference_mode():
                model.eval()
                preds = model(t.tensor([[1,2,3,4,5,6]]).to(device))
                preds = preds.argmax(dim=-1)

                random_case = t.randint(0, 9, (6, )).unsqueeze(dim=0)
                random_preds = model(random_case.to(device)).argmax(dim=-1)
                random_corrects = random_case.flip(dims=[0])
                accuracy = (random_preds == random_corrects).sum() / len(random_preds)
                model.train()

            progress_bar.set_description(f"Epoch = {epoch}, Preds = {preds.squeeze()}, Loss = {loss.item():.4f}, Accuracy = {accuracy}")

    print(f"Saving model to: {MODEL_FILENAME}")
    t.save(model, MODEL_FILENAME)
    return loss_list, accuracy_list

loss_list, accuracy_list = train_transformer(trainloader, epochs, loss_fn)

fig = px.line(y=loss_list, template="simple_white")
fig.update_layout(title="Cross entropy loss on number sequences", yaxis_range=[0, max(loss_list)])
fig.show()

  0%|          | 0/3907 [00:00<?, ?it/s]

torch.Size([256, 6, 10])
tensor([[  8.1746,  11.3028,   1.3904,  ..., -20.4422,   3.7496,   3.2967],
        [-24.8902,  72.7756,   5.9577,  ...,  -7.7274,   9.7350,   4.7106],
        [-11.1063,   9.9323,  13.3041,  ...,  -6.6385,  17.0234, -11.5643],
        ...,
        [ 56.0778,  15.0765,   1.8483,  ..., -14.6076,   9.3937,  11.3468],
        [  0.1942,  15.8409,   1.3735,  ..., -22.7217,  -2.8546,   5.4725],
        [  1.8736,  16.6649,   1.8090,  ..., -17.5331,   0.3683,  10.3631]],
       grad_fn=<ReshapeAliasBackward0>)
torch.Size([256, 6])
torch.Size([256, 6, 10])
tensor([[-13.9192,  62.8246,  11.6134,  ...,  -1.9027,  18.4996,  -2.3679],
        [-11.1304,  22.0089,  16.5858,  ...,  -6.6018,  18.3883, -13.0610],
        [-13.9364,  62.5826,   0.6895,  ...,  -7.0561,  20.1084,  -8.5555],
        ...,
        [ -0.1305,  33.4511,   2.9367,  ...,  -0.6622,  20.4960,  10.8302],
        [ -8.0634,  23.6785,   9.7192,  ...,  -3.5026,  17.3290,   7.3550],
        [ -5.3289,  22.648

KeyboardInterrupt: 